In [1]:
from time import sleep
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

# pull in the customlibrary
%run pyaes.ipynb

In [2]:
led0 = base.leds[0] #Corresponds to LED LD0
led1 = base.leds[1] #Corresponds to LED LD1
led2 = base.leds[2] #Corresponds to LED LD2
led3 = base.leds[3] #Corresponds to LED LD3

In [3]:
sw0 = base.switches[0] #Corresponds to SW0
sw1 = base.switches[1] #Corresponds to SW1

In [4]:
btn0 = base.buttons[0] # button 0 hopefully
btn1 = base.buttons[1]
btn2 = base.buttons[2]
btn3 = base.buttons[3]

In [ ]:
print("Welcome to the pynq z2 board test")

Welcome to the pynq z2 board test


In [ ]:
# switch based LED toggling for test
data= ''
key = b'aesEncryptionKey'
while(True):  # All the code below while(True) runs forever
    if sw1.read():
        led2.toggle()
        led3.toggle()
        if btn0.read():
            led1.toggle()
            led0.toggle()
        sleep(2)
        
    # run the input read and encryption
    else:
        if btn0.read():
            data = ""
            key = b'aesEncryptionKey'
        # read in data if the sw0 is on
        elif btn1.read():
            data += input()
            print(f"data to encrypt: {data}")
        elif btn2.read():
            key = input()
            while len(key) < 16:
                key+='0'
            print(f"key: {key}")
        # do the encryption and print if btn1 is pressed
        elif btn3.read():
            plaintext = str.encode(data)

            print(f"Encrypting {plaintext} with key {key}")
            encrypter = Encrypter(AESModeOfOperationECB(key))
            ciphertext = encrypter.feed(plaintext)
            ciphertext += encrypter.feed()

#            print( repr(ciphertext))
            for letter in ciphertext:
                print('{:0>2}'.format(str(hex(letter))[2:]), end='')
            sleep(1)

           
            # We can encrypt one line at a time, regardles of length

            # We can decrypt the cipher text in chunks (here we split it in half)
#             decrypter = pyaes.Decrypter(pyaes.AESModeOfOperationECB(key))
#             decrypted = decrypter.feed(ciphertext[:len(ciphertext) / 2])
#             decrypted += decrypter.feed(ciphertext[len(ciphertext) / 2:])
#             decrypted += decrypter.feed()
    
        
    # button four starts the basic hello world aes test
    if sw0.read() and btn0.read():
        print("SW1 controls whether lights or encryption (on for lights)")
        print("In encryption mode:")
        print("\t BTN0 resets the data")
        print("\t BTN1 orders input of data\n")
        print("\t BTN2 allows key input (padded to 16 characters)")
        print("\t BTN3 does encryption")
              
        print("Sample encryption and help display:")
        key = b'aesEncryptionKey'
        plaintext = b'hello, world0000'

        aes = AESModeOfOperationECB(key)
        ciphertext = aes.encrypt(plaintext)

        print( repr(ciphertext))
        decrypted = aes.decrypt(ciphertext)
        print (f"match? {decrypted == plaintext}")
        sleep(5)
    


hello, world
data to encrypt: hello, world
Encrypting b'hello, world' with key b'aesEncryptionKey'
9f1c7135939287747aa0f2a539739983